In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('../input/foursquare-location-matching/train.csv')
test = pd.read_csv('../input/foursquare-location-matching/test.csv')
poiext=pd.read_csv('../input/points-of-interest-poi-database/poi.csv')


In [ ]:
import math

poiext['latitude']=[math.degrees(lr) for lr in poiext['latitude_radian']]
poiext['longitude']=[math.degrees(lr) for lr in poiext['longitude_radian']]
poiext['addr']=poiext['links'].astype('str')

# Scatterdiagram

normally the error between two points submitted with a GSM can be maximally 60meter, due to the typical sattelite GPS error
to vizualize this, you simply have to scatterplot the data

its very bizar to see margins of error of 150°  or half the world round..;
this is IMPOSSIBLE with the GPS coordinates submitted automatically...
the real problem should be not to find errors the world round, but errors in de order of margin of 60meter


In [ ]:
sample=train[['point_of_interest','latitude','longitude','id','name']].sort_values(['latitude'])[:10000].groupby(['point_of_interest','latitude','longitude']).max().reset_index()
sample=sample.merge(sample,how='left',left_on='point_of_interest',right_on='point_of_interest',suffixes=('','_y'))
sample['latd']=sample['latitude']-sample['latitude_y']
sample['lond']=sample['longitude']-sample['longitude_y']
sample.plot.scatter(x='latd',y='lond')

In [ ]:
sample

In [ ]:
#sample[sample.latd>5][:50].merge(poiext,how='left',left_on=['lat','lon'],right_on=['lat','lon'])

In [ ]:
sample.sort_values('lond')

# Merge with external DB

In [ ]:
poiext['lat']=np.round(poiext['latitude']*10)/10
poiext['lon']=np.round(poiext['longitude']*10)/10
sample['lat']=np.round(sample['latitude_y']*10)/10
sample['lon']=np.round(sample['longitude_y']*10)/10


In [ ]:
sample[sample.latd>4.5][:60].merge(poiext,how='left',left_on=['lat','lon'],right_on=['lat','lon'])

In [ ]:
sample['lat']=np.round(sample['latitude']*10)/10
sample['lon']=np.round(sample['longitude']*10)/10

sample[sample.lond>4][:50].merge(poiext,how='left',left_on=['lat','lon'],right_on=['lat','lon'])

# lets verify all POI's


In [ ]:
train['lat']=np.round(train['latitude']*10)/10
train['lon']=np.round(train['longitude']*10)/10



In [ ]:
POIver=train.merge(train[['point_of_interest','latitude','longitude','lat','lon']],how='left',left_on='point_of_interest',right_on='point_of_interest')
POIver['coorOK']=(POIver['lat_x']==POIver['lat_y'])&(POIver['lon_x']==POIver['lon_y'])
POIver['latd']=POIver['lat_x']-POIver['lat_y']
POIver['lond']=POIver['lon_x']-POIver['lon_y']

POIver

In [ ]:
POIOK=POIver[POIver['coorOK']==True]
POINOK=POIver[POIver['coorOK']==False]

len(POIOK),len(POINOK)

so a majority of data is available and verified, thats the good news...
but lets have a look at these errors...


In [ ]:
POINOK.groupby(['id','latitude_x','longitude_x','name','country','city']).max().reset_index().merge(poiext,how='left',left_on=['lat_x','lon_x'],right_on=['lat','lon']).sort_values('latd')[:50]

In [ ]:
POINOK.groupby(['id','latitude_y','longitude_y','name','country','city']).max().reset_index().merge(poiext,how='left',left_on=['lat_y','lon_y'],right_on=['lat','lon']).sort_values('latd')[:50]

![](https://live-production.wcms.abc-cdn.net.au/50aea389c0064404e68525dc5d6125da?impolicy=wcms_crop_resize&cropH=1331&cropW=1996&xPos=52&yPos=26&width=862&height=575)
# there is an eifel tower in Melbourne... 

but the Name seems to be wrongly connected to Paris...
so most connections with POI names are made matching the names.. but are simply wrong on one side..
this is bad work, and that makes the verification target so low we obtain only 90% correctness


